In [105]:
EXPERIMENT_IDENTIFIER = "completed"

In [64]:
N = 5

In [65]:
!ls ../data/annotated_100_ethnic_article_against_5_metrics.csv

ls: cannot access '../data/annotated_100_ethnic_article_against_5_metrics.csv': No such file or directory


In [66]:
import pandas as pd
ethnic_df = pd.read_csv("../data/annotated_100_ethnic_article_against_5_metrics.csv")
ethnic_df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1, inplace=True)
ethnic_df

In [67]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

### Loading model

In [6]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [82]:
from openai import OpenAI
import json

client = OpenAI()

def get_response(news_article):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": news_article}
        ]
    )
    return completion.choices[0].message.content
    
def post_process_chatgpt_turbo_3_5_response_to_json(text):
    json_string = text.replace("```", "").replace("json","").replace("\n","").replace("'", '"').replace("\t","").replace('"s ', '\'s ')
    #print(f"Midway: {json_string}")
    json_output = json.loads(json_string)
    return json_output

In [93]:
with open('prompt.txt', 'r') as file:
    prompt = file.read()
print(prompt)

You are a Bangla news article analyzer. Given an article, analyze the following aspects:

    1. Genre: Identify the genre(s) of the article. Choose from: Politics, Crime, International Affairs, Business & Economy, Sports, Culture & Entertainment, Jobs, Lifestyle, Health, Environment, Local news, Education, Technology, Religion, Science, Protests and Social Movements, Govt Actions.
    2. Journalism Style: Determine the journalism style used in the article. Choose from: Straight, Investigative, Commentary.
    3. indigenous People Representation:
        - Identify if indigenous people are featured in the article.
        - If yes, categorize their role as one of the following: Subject, Spokesperson, Expert or Commentator, Personal Experience, Popular Opinion, Eyewitness.
        - If indigenous people are only mentioned in passing, return false. Otherwise, return true.
    4. Sentiment: Assess the sentiment of the article towards indigenous people. Choose from: Positive, Negative, Neu

In [94]:
from tqdm import tqdm

articles = ethnic_df["content"].values.tolist()
responses = []
for article in tqdm(articles[:N]):
    responses.append(get_response(article[:10000]))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:59<00:00,  1.80s/it]


In [95]:
responses[:5]

['{\n  "genre": ["Crime", "Politics"],\n  "journalism_style": "Straight",\n  "featured": "Subject",\n  "is_indigenous": true,\n  "sentiment": "Negative",\n  "agenda_setting": "Violence",\n  "framing": "Resignation",\n  "priming": "Assassination",\n  "mobilizing": "Arrests"\n}',
 '{\n  "genre": ["Crime"],\n  "journalism_style": "Straight",\n  "featured": "Subject",\n  "is_indigenous": true,\n  "sentiment": "Negative",\n  "agenda_setting": "Conflict",\n  "framing": "Violence",\n  "priming": "Armed",\n  "mobilizing": "Clash"\n}',
 '{\n  "genre": ["Crime"],\n  "journalism_style": "Straight",\n  "featured": "Subject",\n  "is_indigenous": false,\n  "sentiment": "Negative",\n  "agenda_setting": "Immigration",\n  "framing": "Detainment",\n  "priming": "Border Security",\n  "mobilizing": "Enforcement"\n}',
 '{\n  "genre": ["Culture & Entertainment", "Local news"],\n  "journalism_style": "Straight",\n  "featured": "",\n  "is_indigenous": false,\n  "sentiment": "Positive",\n  "agenda_setting": "L

In [96]:
true_labels = [1,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,1,1,1,0,
               1,1,1,0,1,0,0,0,1,1,1,1,0,1,0,1,
               0,1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1]

In [98]:
ethnic_sample_df = ethnic_df.iloc[:N].copy()
ethnic_sample_df["Genre"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["genre"] for response in responses]
ethnic_sample_df["Journalism Style"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["journalism_style"] for response in responses]
ethnic_sample_df["Featured"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["featured"] for response in responses]
ethnic_sample_df["Is_ethnic"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["is_indigenous"] for response in responses]
ethnic_sample_df["Sentiment"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["sentiment"] for response in responses]
ethnic_sample_df["Agenda Setting"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["agenda_setting"] for response in responses]
ethnic_sample_df["Framing"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["framing"] for response in responses]
ethnic_sample_df["Priming"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["priming"] for response in responses]
ethnic_sample_df["Mobilizing"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["mobilizing"] for response in responses]

ethnic_sample_df.head()

,content,Genre,Journalism Style,Featured,Is_ethnic,Sentiment,Agenda Setting,Framing,Priming,Mobilizing
0,রাঙ্গামাটিতে আওয়ামী লীগে ভর্তি আতঙ্ক ছড়িয়ে পড়ে...,"[Crime, Politics]",Straight,Subject,True,Negative,Violence,Resignation,Assassination,Arrests
1,রাঙামাটির লংগদু উপজেলায় সশস্ত্র দুই গ্রুপের সং...,[Crime],Straight,Subject,True,Negative,Conflict,Violence,Armed,Clash
2,ব্রাহ্মণবাড়িয়ার আখাউড়া উপজেলায় ভারতের ত্রিপুরা...,[Crime],Straight,Subject,False,Negative,Immigration,Detainment,Border Security,Enforcement
3,ভৌগলিক সীমারেখা ভুলে কেবল ভাষার টানে দুই বাংলা...,"[Culture & Entertainment, Local news]",Straight,,False,Positive,Language,Unity,Cultural Exchange,Unity
4,চাঁদাবাজির ১১ লাখ টাকাসহ পার্বত্য চট্টগ্রাম জন...,[Crime],Straight,Subject,True,Negative,Corruption,Arrest,Illegal Activity,Enforcement


In [99]:
predicted_labels = ethnic_sample_df["Is_ethnic"].values.tolist()
predicted_labels = [1 if item == True else 0 for item in predicted_labels]
predicted_labels[:10]

[1, 1, 0, 0, 1, 1, 1, 1, 1, 1]

In [100]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Calculating precision, recall, accuracy, and F1 score
precision = precision_score(true_labels[:N], predicted_labels)
recall = recall_score(true_labels[:N], predicted_labels)
accuracy = accuracy_score(true_labels[:N], predicted_labels)
f1 = f1_score(true_labels[:N], predicted_labels)

# Print the results
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Precision: 0.8125
Recall: 0.8253968253968254
Accuracy: 0.77
F1 Score: 0.8188976377952756


In [106]:
ethnic_sample_df.to_csv(f"../data/ethnic_{N}_gpt_3.5_turbo_{EXPERIMENT_IDENTIFIER}.csv")

In [107]:
import joblib
joblib.dump(responses, f"../data/ethnic_{N}_gpt3.5_turbo_{EXPERIMENT_IDENTIFIER}.joblib")

['../data/ethnic_100_gpt3.5_turbo_final_indigenous_prompt.joblib']

In [108]:
ethnic_sample_df["Is_ethnic"].values.tolist()

[True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True]

### MISC

In [11]:
abhi_labels = [1,1,0,0,1,1,1,0,1,1,0,0,1,1,1,1,1,1,0,
               1,1,0,0,1,0,1,0,1,1,0,1,1,0,0,0,1,1,1,
               1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,0,1,1,
               1,1,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,
               0,0,1,0,1,
               1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,]
sharif_labels = [ 1,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,1,1,1,0,
                 1,1,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,0,1 ]

In [14]:
import numpy as np
def annotator_agreement_score(labels1, labels2):
    # Calculate agreement score
    total_labels = len(labels1)
    agreements = np.sum(np.array(labels1) == np.array(labels2))
    agreement_score = agreements / total_labels

    # Find conflicting posts
    conflict_indices = [i for i in range(total_labels) if labels1[i] != labels2[i]]
    conflicting_posts = [(i, labels1[i], labels2[i]) for i in conflict_indices]

    return agreement_score, len(conflicting_posts)
annotator_agreement_score(abhi_labels, sharif_labels)

(np.float64(0.75), 25)

In [80]:
from collections import Counter

counts = Counter(true_labels)
print("Yes count:", counts[1])
print("No count:", counts[0])

Yes count: 63
No count: 37


In [45]:
ethnic_sample_df.iloc[2].values.tolist()

[np.int64(2),
 'গাইবান্ধার গোবিন্দগঞ্জের রংপুর চিনিকলের সাহেবগঞ্জ ইক্ষু খামারে দেয়া কাঁটাতারের বেড়ার পিলার ভাংচুর করেছে দুর্বৃত্তরা।সোমবার রাতে খামার সংলগ্ন জয়পুর পাড়ার দক্ষিণ অংশে দেয়া ২৬টি পিলার ভাংচুর করা হয়।খামারের জমি সাঁওতালদের কাছ থেকে দখলমুক্ত করার পর মিল কর্তৃপক্ষ এই কাঁটাতারের বেড়া দেয়। এরপর সোমবার রাতে সাঁওতাল পল্লী সংলগ্ন খামারের বেড়ার পিলার ভাংচুর করে দুর্বৃত্তরা।রংপুর সুগার মিলের ব্যবস্থাপনা পরিচালক আব্দুল আউয়াল জানান, দুস্কৃতকারীরা রাতের আধারে খামারের পশ্চিম অংশে সাঁওতাল পল্লী জয়পুর পাড়া সংলগ্ন ২৬টি পিলার ভাংচুর করে। এ ঘটনায় ১৩ জনের নাম উল্লেখ করে খামারের উপ-ব্যবস্থাপক মো. হুমায়ন কবির মঙ্গলবার রাতে গোবিন্দগঞ্জ থানায় একটি জিডি করেন।গোবিন্দগঞ্জ থানার ওসি সুব্রত কুমার সরকার বিষয়টি নিশ্চিত করে বলেন, ঘটনার বিষয়টি খতিয়ে দেখা হচ্ছে।গত ৬ নভেম্বর আদিবাসী সাঁওতালদের সঙ্গে আইনশৃংখলা বাহিনী চিনিকলের শ্রমিক-কর্মচারীদের সংঘর্ষের পর ইক্ষু খামার থেকে সাঁওতাল বসতি উচ্ছেদ, লুটপাট, গুলিতে ৩ সাঁওতাল নিহত হওয়ার ঘটনায় সেখানে এখনও অস্থিরতা রয়েছে।',
 'Crime',
 'Investigative',
 'Victim',
 np.Tr

In [78]:
ethnic_sample_df = ethnic_df.sample(n=100, random_state=3)

In [79]:
ethnic_sample_df.to_csv("../data/annotated_100_ethnic_article_against_5_metrics.csv")